# Describe the Data I will be using 

I am using the Foursquare API to access venue data across the various cities and neighborhoods. For each city-- Philadelphia, PA; Charlotte, NC; and Scottsdale, AZ-- I am constructing its own Pandas dataframe to preprocess the data. I plan to store preliminary search results in sub-dataframes.  From there I will manipulate the data until it is usable and then concatinate or merge it into one dataframe for each city.

I will manually input the address of the office location.  The data I am searching for includes name, location and category type for each of the following:

   - churches
   - grocery stores
   - gass stations
   - parks
   - shopping mall
   - Thai restaurants
   - museums
   - zoo
   - golf courses
   - grade schools
   - library branches
 
Once the data has been scrubbed and is in a usable format, I will plot it by run it through a K-means process to visually analyze the data on a map.

# Import necessary libraries

Some of the libraries that I anticipte using.

In [1]:
import numpy as np # to handle data in a vectorized manner
import pandas as pd # for data analsysis
import json #  handle JSON files
#!pip install geopy #if needed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # to plot
import matplotlib.colors as colors #to plot
from sklearn.cluster import KMeans # to use k-means clustering
#!conda install -c conda-forge folium=0.5.0 --yes # if needed
import folium # plot on maps

print('done')

done


# Example of the Data

In [2]:
# Foursquare credentials and version

CLIENT_ID = 'GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO' # your Foursquare ID
CLIENT_SECRET = 'WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

In [3]:
address = '100 Vanguard Malvern, PA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


40.0521503 -75.505871


In [4]:
search_query = 'church'
radius = 16093
print('done')

done


In [5]:
url = url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOTROBRRHDUOVY4VLPYSQYYXKCL2J13WBPYCREWYGYX1LNMO&client_secret=WGKCPZRXFNFE0AAFT4EHSG4UHKZEBLHO0GX2KNYQ2XKI3CSD&ll=40.0521503,-75.505871&v=20180604&query=church&radius=16093&limit=50'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1359dbea795307ab53ea84'},
 'response': {'venues': [{'id': '4e2ef003e4cdb5a70d631cec',
    'name': 'The Church of Saint Monica',
    'location': {'address': '63 Main Ave',
     'crossStreet': 'at First Ave',
     'lat': 40.04608705725427,
     'lng': -75.44035900501834,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.04608705725427,
       'lng': -75.44035900501834}],
     'distance': 5623,
     'postalCode': '19312',
     'cc': 'US',
     'city': 'Berwyn',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['63 Main Ave (at First Ave)',
      'Berwyn, PA 19312',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d132941735',
      'name': 'Church',
      'pluralName': 'Churches',
      'shortName': 'Church',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_church_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595103912',
    'has

In [7]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4e2ef003e4cdb5a70d631cec,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,The Church of Saint Monica,v-1595103912,NaN
1,"[{'id': '4bf58dd8d48988d1a8941735', 'name': 'G...",False,4c3de78f15cb1b8d198f7cdc,1001 E Lincoln Hwy,US,Exton,United States,NaN,7889,"[1001 E Lincoln Hwy, Exton, PA 19341, United S...","[{'label': 'display', 'lat': 40.03273681214128...",40.032737,-75.594901,19341,PA,Church Farm School,v-1595103912,NaN
2,[],False,5577e96b498e7a29340213b3,NaN,US,Paoli,United States,NaN,1722,"[Paoli, PA, United States]","[{'label': 'display', 'lat': 40.0415981, 'lng'...",40.041598,-75.491087,NaN,PA,Church of the Good Samaritan,v-1595103912,NaN
3,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4b7c9e6ef964a520189e2fe3,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,Church of the Good Samaritan,v-1595103912,NaN
4,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",False,4d15785d6d103704171d2abd,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,St Mary Of The Assumption Church,v-1595103912,NaN


In [8]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Church of Saint Monica,Church,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,4e2ef003e4cdb5a70d631cec
1,Church Farm School,General College & University,1001 E Lincoln Hwy,US,Exton,United States,NaN,7889,"[1001 E Lincoln Hwy, Exton, PA 19341, United S...","[{'label': 'display', 'lat': 40.03273681214128...",40.032737,-75.594901,19341,PA,4c3de78f15cb1b8d198f7cdc
2,Church of the Good Samaritan,None,NaN,US,Paoli,United States,NaN,1722,"[Paoli, PA, United States]","[{'label': 'display', 'lat': 40.0415981, 'lng'...",40.041598,-75.491087,NaN,PA,5577e96b498e7a29340213b3
3,Church of the Good Samaritan,Church,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,4b7c9e6ef964a520189e2fe3
4,St Mary Of The Assumption Church,Church,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,4d15785d6d103704171d2abd
5,Church of the Saviour,Church,651 N Wayne Ave,US,Wayne,United States,at Woodsworth Ct,9811,"[651 N Wayne Ave (at Woodsworth Ct), Wayne, PA...","[{'label': 'display', 'lat': 40.05359304988622...",40.053593,-75.390735,19087,PA,4b531f05f964a5207c8f27e3
6,"Saint Luke Lutheran Church - Devon, PA",Church,203 N Valley Forge Rd,US,Devon,United States,Conestoga Rd.,7212,"[203 N Valley Forge Rd (Conestoga Rd.), Devon,...","[{'label': 'display', 'lat': 40.05066761991265...",40.050668,-75.421251,19333,PA,4ec0362b30f82a2e13521d83
7,St. David's Episcopal Church,Church,762 Valley Forge Rd,US,Wayne,United States,btwn Church Rd & Maplewood Rd,9127,[762 Valley Forge Rd (btwn Church Rd & Maplewo...,"[{'label': 'display', 'lat': 40.02697026100816...",40.026970,-75.403951,19087,PA,4e19d50dd4c062b044f53f58
8,St. Ann's Church,Church,3rd ave,US,Phoenixville,United States,main st,8274,"[3rd ave (main st), Phoenixville, PA 19460, Un...","[{'label': 'display', 'lat': 40.12622427575029...",40.126224,-75.513958,19460,PA,4c5590b2479fc9287aa70b95
9,"First Church Of Christ, Scientist",Church,NaN,US,Phoenixville,United States,NaN,1459,"[Phoenixville, PA, United States]","[{'label': 'display', 'lat': 40.03923316394582...",40.039233,-75.508792,NaN,PA,539e0497498e71ce2a63cea3


In [9]:
dataframe_filtered.shape

(50, 15)

# Example of sub-dataframe

In [12]:
df_church_pa = dataframe_filtered[dataframe_filtered.categories == 'Church']
df_church_pa.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Church of Saint Monica,Church,63 Main Ave,US,Berwyn,United States,at First Ave,5623,"[63 Main Ave (at First Ave), Berwyn, PA 19312,...","[{'label': 'display', 'lat': 40.04608705725427...",40.046087,-75.440359,19312,PA,4e2ef003e4cdb5a70d631cec
3,Church of the Good Samaritan,Church,Lancaster Ave,US,Paoli,United States,NaN,1807,"[Lancaster Ave, Paoli, PA 19301, United States]","[{'label': 'display', 'lat': 40.04091559619378...",40.040916,-75.490552,19301,PA,4b7c9e6ef964a520189e2fe3
4,St Mary Of The Assumption Church,Church,199 St. Marys St.,US,Phoenixville,United States,Dayton St.,9852,"[199 St. Marys St. (Dayton St.), Phoenixville,...","[{'label': 'display', 'lat': 40.14046, 'lng': ...",40.140460,-75.513549,19460,PA,4d15785d6d103704171d2abd
5,Church of the Saviour,Church,651 N Wayne Ave,US,Wayne,United States,at Woodsworth Ct,9811,"[651 N Wayne Ave (at Woodsworth Ct), Wayne, PA...","[{'label': 'display', 'lat': 40.05359304988622...",40.053593,-75.390735,19087,PA,4b531f05f964a5207c8f27e3
6,"Saint Luke Lutheran Church - Devon, PA",Church,203 N Valley Forge Rd,US,Devon,United States,Conestoga Rd.,7212,"[203 N Valley Forge Rd (Conestoga Rd.), Devon,...","[{'label': 'display', 'lat': 40.05066761991265...",40.050668,-75.421251,19333,PA,4ec0362b30f82a2e13521d83


In [13]:
df_church_pa.shape

(41, 15)